In [24]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt

In [25]:
tables=camelot.read_pdf("axis03.pdf",flavor='stream' ,pages = "all",edge_tol=500)            
if len(tables) !=0 :
    df=pd.DataFrame();    tmp=pd.DataFrame(); 
    for i in range(len(tables)) :
        tmp=tables[i].df
        if (tmp.shape[1] <= 5):
            pass
        elif (tmp.shape[1] == 8):   
            df=pd.concat([df,tmp]).drop_duplicates().reset_index(drop=True)
        elif (tmp.shape[1] == 6):
            try:
                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
            except:
                try:
                    idx=[ c for c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                except:
                    print("\nAxis-Column headers missing")
            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
            tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
            if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                tmp.columns = ["A","B","C","D","E","F"]
                tmp.reset_index(drop = True, inplace=True)
                tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                result = pd.concat([tmp1, tmp], axis=1, sort=False)
                result.drop('A' ,axis=1, inplace=True)
                result["Y"] = result["Z1"]
                for i,j in enumerate(result["Y"]):
                    if re.search(r'(\d+-\d+-\d+)',j):
                        result.at[i, 'Y'] = ""
                        pass
                    elif re.search(r'^\s*$',j):
                        result.at[i, 'Y'] = ""
                        pass
                    else:
                        pass
                        
                for i,j in enumerate(result["Z1"]):
                    if re.search(r'(\d+-\d+-\d+)',j):
                        pass
                    elif re.search(r'^\s*$',j):
                        result.at[i, 'Z1'] = ""
                        pass
                    else:
                        result.at[i, 'Z1'] = ""
                        pass 
                result.fillna(value=pd.np.nan, inplace=True)
                result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                result.drop(['Z3','Y'] ,axis=1, inplace=True)
                result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                result.columns = [0,1,2,3,4,5,6,7]
                df=pd.concat([df,result]).drop_duplicates().reset_index(drop=True)
        else:
            try:
                try:
                    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                    df.columns=df.iloc[idx] ; df=df.iloc[idx:,:] ; df.reset_index(drop=True,inplace=True)
                except:
                    try:
                        idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                    except:
                        print("\nAxis-Column headers missing"); pass
                tmp.drop("",axis = 1,inplace = True)
                tmp.columns = [0,1,2,3,4,5,6,7]
                df=pd.concat([df,tmp]).drop_duplicates().reset_index(drop=True)
            except:pass
           
    print("Completed 1")
print("Completed 2")

c:\users\mudracircle\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Completed 1
Completed 2


In [3]:
df.to_csv("parsed_01.csv", index=0)

In [4]:
df.head()

,0,1,2,3,4,5,6,7
0,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance,Branch Name
1,,,OPENING BALANCE :,,,,25184.55,
2,,,,,,,,ATM
3,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,,5053.95,CR,30238.50,RECONCILATION
4,,,,,,,,CENTRE


In [5]:
try:
    idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
    print("1st attempt")
except:
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
    except:
        print("\nAxis-Column headers missing"); pass
    
df.head()

1st attempt


,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance,Branch Name
0,,,OPENING BALANCE :,,,,25184.55,
1,,,,,,,,ATM
2,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,,5053.95,CR,30238.50,RECONCILATION
3,,,,,,,,CENTRE
4,,,,,,,,ZAMROODPUR


In [6]:
df.to_csv("parsed_02.csv", index=0)

In [7]:
try:
    df = df.drop(["Init."], axis=1)
except:
    try:
        df = df.drop(["Branch Name"], axis=1)
    except:pass
    
df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('opening balance|transaction total|closing balance').any(), axis=1) ==True].index)]
df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)

def abc(a):
    if type(a) ==str:
        if len(a.split(' '))==2:
            z=a.split(' ')[1]
        else:
            z=a.split(' ')[0]
    else :
        z=a
    return z

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper()][0]
    df[bal]=df[bal].apply(lambda x: abc(x))
except:
    print("\nBalance Column Missing")

In [8]:
df.head()

,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance
1,,,,,,,
2,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,,5053.95,CR,30238.50
3,,,,,,,
4,,,,,,,
5,06-12-2018,06-12-2018,ECOM PUR/PTM*PAYTM/NOIDA/061218/09:18,,7853.25,DR,22385.25


In [9]:
df.to_csv("parsed_03.csv", index=0)

In [10]:
df.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [11]:
type(df["Transaction Particulars"][15])

float

In [12]:
# df.head(16)

In [13]:
df.dropna(axis=0, how='all',inplace=True)

In [14]:
df.head()

,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance
2,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,NaN,5053.95,CR,30238.50
5,06-12-2018,06-12-2018,ECOM PUR/PTM*PAYTM/NOIDA/061218/09:18,NaN,7853.25,DR,22385.25
7,07-12-2018,07-12-2018,IMPS/P2A/834111246344/50100096080290HDFC0004108/,NaN,12005.90,DR,10379.35
8,07-12-2018,07-12-2018,UPI/P2A/834184872222/YBL5e634a07d3c24593ba2586b7,NaN,1700.00,CR,12079.35
9,14-12-2018,14-12-2018,ECOM PUR/One97 Communi/Noida/141218/12:02,NaN,480.00,DR,11599.35


In [15]:
df.reset_index(drop=True,inplace=True)

In [16]:
df.to_csv("parsed_04.csv", index=0)

In [17]:
def isnan(value):
    try:
        import math
        return math.isnan(float(value))
    except:
        return False

In [18]:
for j,i in enumerate(df["Transaction Particulars"]):
    if isnan(i) == True:
        df["Transaction Particulars"][j] = df["Transaction Particulars"][j-1]+df["Transaction Particulars"][j+1]
    else:
        pass

In [22]:
try:
    df.dropna(subset=['Tran Date'], inplace = True)
except:
    df.dropna(subset=['Value Date'], inplace = True)

In [23]:
df.to_csv("parsed_05.csv",index=0)

In [ ]:
df = df.dropna(how='all',axis=0)
df = df.reset_index(drop=True)

In [ ]:
df.to_csv("parsed.csv", index=0)